# Symmetric Funnel Decomposition Analysis - Demo

This notebook demonstrates the **symmetric (order-independent)** funnel decomposition analysis workflow.

## Key Difference from Hierarchical Approach

**Hierarchical decomposition** uses a sequential waterfall where each effect is calculated step-by-step using values from previous steps. This makes it order-dependent.

**Symmetric decomposition** uses a midpoint methodology where all effects are calculated independently using average values from both periods. This makes it order-independent.

## Workflow
1. Load booking funnel data
2. Calculate symmetric decomposition effects
3. Visualize results with waterfall charts
4. View detailed breakdowns by dimension

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import symmetric_decomposition_calculator
import visualization_engine

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 100

## 1. Load Mock Data

The dataset contains 24 months of booking funnel data with:
- **Dimensions**: FICO bands (High/Med/Low/Null), offer competition tier, product line
- **Funnel metrics**: Applications, approval rates, booking rates
- **Segments**: 24 segments per month (4 FICO bands × 3 offer tiers × 2 product lines)

In [ ]:
# Load data
data_path = Path.cwd().parent / 'data' / 'funnel_data_mock_v2.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

print(f"Loaded {len(df)} rows of data")
print(f"\nDate range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Unique months: {df['month_begin_date'].nunique()}")
print(f"Segments per month: {len(df) // df['month_begin_date'].nunique()}")

# Show sample data
print("\nSample data:")
df.head(10)

## 2. Calculate Symmetric Decomposition

We'll decompose the booking change from **June 2023 → June 2024** (Year-over-Year) into 6 effects:

### Symmetric Methodology (Order-Independent)

All effects use **average values** from both periods:

1. **Volume Effect**: `ΔA × p_avg × conversion_avg`
   - Change in total apps × average mix × average conversion

2. **Mix Effect**: `A_avg × Δp × conversion_avg`
   - Average apps × change in mix × average conversion

3. **Straight Approval Effect**: `A_avg × p_avg × Δr_str × b_str_avg`
   - Change in straight approval rates

4. **Conditional Approval Effect**: `A_avg × p_avg × Δr_cond × b_cond_avg`
   - Change in conditional approval rates

5. **Straight Booking Effect**: `A_avg × p_avg × r_str_avg × Δb_str`
   - Change in straight booking rates

6. **Conditional Booking Effect**: `A_avg × p_avg × r_cond_avg × Δb_cond`
   - Change in conditional booking rates

Where `_avg = (value_1 + value_2) / 2`

In [ ]:
# Calculate symmetric decomposition for June 2023 → June 2024 (Year-over-Year)
results = symmetric_decomposition_calculator.calculate_decomposition(
    df=df,
    date_a='2023-06-01',
    date_b='2024-06-01',
    lender='ACA'
)

print("Symmetric Decomposition Results:")
print(f"  Method: {results.metadata['method']}")
print(f"  Period 1: {results.metadata['date_a']}")
print(f"  Period 2: {results.metadata['date_b']}")
print(f"  Period 1 bookings: {results.metadata['period_1_total_bookings']:,.0f}")
print(f"  Period 2 bookings: {results.metadata['period_2_total_bookings']:,.0f}")
print(f"  Delta bookings: {results.metadata['delta_total_bookings']:+,.0f}")
print(f"\n  Number of segments: {results.metadata['num_segments']}")

## 3. View Summary Table

The summary shows aggregate impact of each effect using the symmetric (order-independent) methodology:

In [ ]:
results.summary

## 4. Create Waterfall Grid

The waterfall grid shows:
- **Top-left**: Overall aggregate waterfall
- **Top-right**: Breakdown by FICO bands (High/Med/Low/Null)
- **Bottom-left**: Breakdown by Offer Comp Tier (solo/multi_best/multi_other)
- **Bottom-right**: Breakdown by Product Line (Used/VMax)

**Color scheme**:
- Green shades = positive contributions
- Red shades = negative contributions
- Gray = Start/End bars

In [ ]:
# Create waterfall grid
fig = visualization_engine.create_waterfall_grid(
    summary=results.summary,
    segment_detail=results.segment_detail,
    lender='ACA'
)

plt.show()

## 5. Detailed Breakdowns by Dimension

For each dimensional waterfall, we can view the exact contribution of each dimension value:

In [ ]:
# Print detailed breakdowns
visualization_engine.print_waterfall_breakdowns(fig)

## 6. Dimension Drilldown Charts

We can create horizontal bar charts showing the impact of each effect broken down by a specific dimension.

### By FICO Band

In [ ]:
# Create FICO band drilldown
fig_fico = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='fico_bands',
    lender='ACA'
)

plt.show()

### By Offer Comp Tier

In [ ]:
# Create Offer Comp Tier drilldown
fig_comp = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='offer_comp_tier',
    lender='ACA'
)

plt.show()

### By Product Line

In [ ]:
# Create Product Line drilldown
fig_prod = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='prod_line',
    lender='ACA'
)

plt.show()

## 7. Segment-Level Detail

For detailed analysis, we can examine the segment-level breakdown:

In [ ]:
# Show segment detail (first 10 segments)
results.segment_detail.head(10)

## 8. Compare Symmetric vs Hierarchical (Optional)

To understand the difference between symmetric and hierarchical decomposition, you can run both and compare:

```python
import hier_decomposition_calculator

# Calculate hierarchical decomposition
results_hier = hier_decomposition_calculator.calculate_decomposition(
    df=df,
    date_a='2023-06-01',
    date_b='2024-06-01',
    lender='ACA'
)

# Compare summaries
comparison = pd.merge(
    results.summary,
    results_hier.summary,
    on='effect_type',
    suffixes=('_symmetric', '_hierarchical')
)
comparison['difference'] = comparison['booking_impact_symmetric'] - comparison['booking_impact_hierarchical']
comparison
```

## 9. Export Results (Optional)

Results can be exported using pandas if needed:

In [ ]:
# Export functionality
# results.summary.to_csv('symmetric_summary.csv', index=False)
# results.segment_detail.to_csv('symmetric_segment_detail.csv', index=False)

## Summary

The **symmetric decomposition** provides an order-independent analysis of booking changes by:

1. Using average values from both periods for all calculations
2. Calculating all effects independently (no sequential dependencies)
3. Ensuring the decomposition results don't depend on the order of effects

This approach is particularly useful when:
- You want a balanced view that doesn't favor one period over another
- Order independence is important for your analysis
- You're comparing multiple decompositions and want consistent methodology

The hierarchical approach may still be preferred when:
- You want to see the incremental impact of each change
- Sequential logic matches your business decision-making process
- You're explaining a specific step-by-step transformation